In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '15'

In [2]:
base_dir = 'archive/shahname.csv'

In [3]:
import csv
text = []
with open(base_dir, 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
        text.append(row[4])


text = [sub.replace('\xa0', ' ') for sub in text]
text = [sub.replace('\u200c', ' ') for sub in text]
text = [sub.replace('آ', 'ا') for sub in text]
text = [sub.replace('َ', '') for sub in text]
text = [sub.replace('ُ', '') for sub in text]
text = [sub.replace('ِ', '') for sub in text]
text = [sub.replace('ة', 'ه') for sub in text]
text = [sub.replace('هٔ', 'ه') for sub in text]
text = [sub.replace('ك', 'ک') for sub in text]
text = [sub.replace('ئ', 'ی') for sub in text]
text = [sub.replace('؛', '') for sub in text]
text = [sub.replace('ّ', '') for sub in text]
text = [sub.replace('ْ', '') for sub in text]
text = [sub.replace('،', '') for sub in text]
text = [sub.replace('ء', '') for sub in text]
text = [sub.replace('«', '') for sub in text]
text = [sub.replace('»', '') for sub in text]
text = [sub.replace('أ', 'ا') for sub in text]
text = [sub.replace(')', '') for sub in text]
text = [sub.replace('(', '') for sub in text]
text = [sub.replace('ؤ', 'و') for sub in text]


text = " ".join(text[0:(len(text) + 1)])

In [4]:
vocab = sorted(set(text))

In [5]:
len(vocab)

37

In [6]:
import numpy as np
import tensorflow as tf

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'به نام خداوند' ---- characters mapped to int ---- > [ 5 28  0 27  4 26  0 10 11  4 29 27 11]


In [9]:
text_as_int

array([ 5, 28,  0, ..., 13, 36, 27])

In [10]:
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ب
ه
 
ن
ا


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
    print("***"*5)

'به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خدا'
***************
'وند کیوان و گردان سپهر فروزنده ماه و ناهید و مهر ز نام و نشان و گمان برتر است نگارنده بر شده پیکر است'
***************
' به بینندگان افریننده را نبینی مرنجان دو بیننده را نیابد بدو نیز اندیشه راه که او برتر از نام و از جا'
***************
'یگاه سخن هر چه زین گوهران بگذرد نیابد بدو راه جان و خرد خرد گر سخن برگزیند همی همان را گزیند که بیند '
***************
'همی ستودن نداند کس او را چو هست میان بندگی را ببایدت بست خرد را و جان را همی سنجد اوی در اندیشه سخته '
***************


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خد'
Target data: 'ه نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خدا'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 5 ('ب')
  expected output: 28 ('ه')
Step    1
  input: 28 ('ه')
  expected output: 0 (' ')
Step    2
  input: 0 (' ')
  expected output: 27 ('ن')
Step    3
  input: 27 ('ن')
  expected output: 4 ('ا')
Step    4
  input: 4 ('ا')
  expected output: 26 ('م')


In [15]:
BATCH_SIZE = 32


dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(32, 100), dtype=tf.int64, name=None), TensorSpec(shape=(32, 100), dtype=tf.int64, name=None))>

In [16]:
vocab_size = len(vocab)
embedding_dim = 25
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model.predict(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

1/1 [==============================] - 0s 330ms/step
(32, 100, 37) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 25)            925       
                                                                 
 gru (GRU)                   (32, None, 1024)          3228672   
                                                                 
 dense (Dense)               (32, None, 37)            37925     
                                                                 
Total params: 3267522 (12.46 MB)
Trainable params: 3267522 (12.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([25, 29, 29, 25,  4, 12, 12, 31,  0,  2,  5,  4, 27, 36, 11, 26,  8,
       18,  5, 24, 20,  1, 21, 33, 30, 33,  3, 30, 18, 12,  6, 22, 12, 31,
       28, 21, 21, 24, 23, 26, 33, 21, 22, 32,  8,  6, 33, 34, 31,  2, 12,
       32, 20, 20, 12,  9, 13, 10,  0, 31,  7,  2,  6,  9, 33, 35, 21, 22,
       13, 10,  5,  3, 30, 15, 28, 22, 22,  3,  4,  1,  6, 27,  3, 36, 10,
        9, 19,  6,  4, 21, 31,  1,  0, 29, 35, 21,  7, 15, 10,  9])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'به نام خداوند جان و خرد کز این برتر اندیشه بر نگذرد خداوند نام و خداوند جای خداوند روزی ده رهنمای خد'

Next Char Predictions: 
 'لوولاذذپ :بانیدمجضبقظ!عژيژ؟يضذتغذپهععقفمژعغچجتژکپ:ذچظظذحرخ پث:تحژگعغرخب؟يسهغغ؟ا!تن؟یخحطتاعپ! وگعثسخح'


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
checkpoint_dir = './training_checkpoints_CharLevel'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
history = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20


I0000 00:00:1717781304.899004   55082 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


792/792 [==============================] - 37s 45ms/step - loss: 2.2083
Epoch 2/20
792/792 [==============================] - 35s 45ms/step - loss: 1.6419
Epoch 3/20
792/792 [==============================] - 36s 45ms/step - loss: 1.4578
Epoch 4/20
792/792 [==============================] - 36s 45ms/step - loss: 1.3580
Epoch 5/20
792/792 [==============================] - 36s 45ms/step - loss: 1.2870
Epoch 6/20
792/792 [==============================] - 36s 45ms/step - loss: 1.2260
Epoch 7/20
792/792 [==============================] - 36s 45ms/step - loss: 1.1714
Epoch 8/20
792/792 [==============================] - 36s 45ms/step - loss: 1.1254
Epoch 9/20
792/792 [==============================] - 36s 45ms/step - loss: 1.0904
Epoch 10/20
792/792 [==============================] - 36s 45ms/step - loss: 1.0709
Epoch 11/20
792/792 [==============================] - 36s 45ms/step - loss: 1.0542
Epoch 12/20
792/792 [==============================] - 36s 45ms/step - loss: 1.0404
Epoch 13/20


In [33]:
training_checkpoints = './training_checkpoints_CharLevel'

In [51]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(training_checkpoints))
model.build(tf.TensorShape([1, None]))

In [52]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (1, None, 25)             925       
                                                                 
 gru_5 (GRU)                 (1, None, 1024)           3228672   
                                                                 
 dense_5 (Dense)             (1, None, 37)             37925     
                                                                 
Total params: 3267522 (12.46 MB)
Trainable params: 3267522 (12.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [59]:
print(generate_text(model, start_string=u"سلام خوبی"))

سلام خوبی بخواهد سپاه و بمرد همیشه تو و هشت در سرت بد که مارم تهی شاه را خواند پیش بدید ان بد و نه شوی نی گوی بدادش می اید و دست بدی به شاهی و از هر دری دیده بود بدو گفت شاها چنین جنگشان کنم هر درین شهر تن پرگزند نباید که یزدان بود نام اوی مران برگزینم بر تاجو سرگرفتی دهد داد و دین و دل را به هر انجمن تو را ای سخنها فراز ارمید همه برفزند از جهان تاج و شمشید جوی سواری و مهرین چو امد بجای به نزد خردمند نزداشت جای گروه روان و جوانان ایران سپاه هم نفرود بپوشد ز جای بزرگان به رنج که یزدان شاهان خورشیدگار ز دینار و دالش گرامی تنست به یزدان کنیم اندر ارندگان نه ان بخت را بس کن ازاد نا اگهی که ارام یاقوت برکش سوار دریغ ایدم و از روزگار بیفشاند کام و اند چو نشچیر تازه به مادر بگوش چنان خست بیشی ز دیبای روم به دیدار چینی درش کرده انداختی دو دیده پر از خون  و رخسموه دید ز اندیشه دل پرددید ن به ایران سپاه اندرست این سخن به زر زاید و هر مهتران ز پیونده روزگار کهن به چنمست همداستان که شد روز بشمر زنی دولبست ولیکن تر از و نان شمشیرزن سر سینه اندر دل هید رای به خشنود یکسر چنان رنج راست بر شاه ایران به